In [ ]:
from asgardpy.analysis import AsgardpyAnalysis
from asgardpy.config import AsgardpyConfig
from asgardpy.data.target import apply_selection_mask_to_models, set_models

from gammapy.maps import Map
from gammapy.modeling.models import Models
from gammapy.modeling import Fit
from gammapy.estimators import FluxPointsEstimator
from gammapy.datasets import Datasets

import matplotlib.pyplot as plt
import numpy as np

import astropy.units as u
import logging

In [ ]:
log = logging.getLogger("test DL4 steps")

In [ ]:
config_file = "/path/to/Config.yaml"

In [ ]:
config = AsgardpyConfig()

In [ ]:
%%time
config_main = config.read(config_file)

In [ ]:
for g in config_main.general:
    print(g)

# Steps mentioned in the main config file

In [ ]:
config_main.general.steps

In [ ]:
for c in config_main.target:
    print(c)

# Datasets informations

In [ ]:
for cc in config_main.dataset3d:
    print(cc)

In [ ]:
for cc in config_main.dataset1d:
    print(cc)

# DL4 step information

In [ ]:
for cc in config_main.fit_params:
    print(cc)

In [ ]:
for cc in config_main.flux_points_params:
    print(cc)

In [ ]:
%%time
analysis = AsgardpyAnalysis(config_main)

In [ ]:
analysis

# Prepare the dataset

In [ ]:
instruments_list_3d = config_main.dataset3d.instruments
print(len(instruments_list_3d), "number of 3D dataset information provided")

config_3d_dataset = instruments_list_3d[0]
print(f"Instrument selected is {config_3d_dataset.name}")

In [ ]:
instruments_list_1d = config_main.dataset1d.instruments
print(len(instruments_list_1d), "number of 1D dataset information provided")

config_1d_dataset = instruments_list_1d[0]
print(f"Instrument selected is {config_1d_dataset.name}")

In [ ]:
key_names = config_3d_dataset.dataset_info.key
print(f"The list of different keys or modes of observations for the selected instrument are {key_names}")

# Run each Dataset nD Generation step

In [ ]:
%%time
analysis.run(["datasets-3d"])

## Display some results

In [ ]:
print(analysis.datasets)

In [ ]:
print(analysis.final_model)

In [ ]:
for m in analysis.final_model:
    print(m)

In [ ]:
print(analysis.instrument_spectral_info)

In [ ]:
print(analysis.dataset_name_list)

In [ ]:
%%time
analysis.run(["datasets-1d"])

## Display the results

In [ ]:
print(analysis.datasets)

In [ ]:
for d in analysis.datasets:
    print(d)

In [ ]:
for m in analysis.final_model:
    print(m)

In [ ]:
print(analysis.instrument_spectral_info)

In [ ]:
for i, en_range in enumerate(analysis.instrument_spectral_info["spectral_energy_ranges"]):
    print(f"The spectral energy range to be used for dataset {analysis.instrument_spectral_info['name'][i]} is {en_range}")

In [ ]:
print(analysis.dataset_name_list)

# Preparing the Datasets for DL4 steps

In [ ]:
analysis.final_model = Models(analysis.final_model)

In [ ]:
# Only updating the target source model with names of all datasets
if len(analysis.final_model) > 1:
    analysis.final_model = apply_selection_mask_to_models(
        list_sources=analysis.final_model,
        target_source=analysis.config.target.source_name,
        roi_radius=analysis.config.target.roi_selection.roi_radius,
        free_sources=analysis.config.target.roi_selection.free_sources,
    )

In [ ]:
for model_ in analysis.final_model:
    print(model_)

## Assigning the models to all the datasets

In [ ]:
analysis.datasets = set_models(
    analysis.config.target,
    analysis.datasets,
    analysis.dataset_name_list,
    models=analysis.final_model,
    target_source_name=analysis.config.target.source_name,
)

In [ ]:
print(analysis.datasets)

In [ ]:
%%time
for d in analysis.datasets:
    print(d)      

# Run the Fit step

In [ ]:
%%time
analysis.run(["fit"])

In [ ]:
print(analysis.fit_result)

In [ ]:
for m in analysis.datasets[0].models:
    print(m)

# Run the Flux Points Estimator step

In [ ]:
%%time
analysis.run(["flux-points"])

In [ ]:
for fp in analysis.flux_points:
    print(fp)

In [ ]:
fermi_fpt = analysis.flux_points[0].to_table(sed_type="e2dnde", formatted=True)
fermi_fpt

In [ ]:
analysis.flux_points[0].plot(sed_type="e2dnde", color="red", label="Fermi-LAT")
analysis.datasets[0].models[config_main.target.source_name].spectral_model.plot(
    sed_type="e2dnde", energy_bounds=[70 * u.MeV,10 * u.TeV]
)
analysis.datasets[0].models[config_main.target.source_name].spectral_model.plot_error(
    sed_type="e2dnde", energy_bounds=[70 * u.MeV,10 * u.TeV]
)

plt.grid(which="both")
plt.legend()

In [ ]:
lst_fpt = analysis.flux_points[1].to_table(sed_type="e2dnde", formatted=True)
lst_fpt

In [ ]:
analysis.flux_points[1].plot(sed_type="e2dnde", color="red", label="LST-1")
analysis.datasets[1].models[0].spectral_model.plot(
    sed_type="e2dnde", energy_bounds=[10 * u.GeV, 10 * u.TeV]
)
analysis.datasets[1].models[0].spectral_model.plot_error(
    sed_type="e2dnde", energy_bounds=[10 * u.GeV, 10 * u.TeV]
)

plt.grid(which="both")
plt.legend()

# Final SED plot with all instruments

In [ ]:
analysis.flux_points[0].plot(sed_type="e2dnde", color="red", label="Fermi-LAT")
analysis.flux_points[1].plot(sed_type="e2dnde", color="green", label="LST-1")

analysis.datasets[1].models[0].spectral_model.plot(
    sed_type="e2dnde", energy_bounds=[70 * u.MeV, 10 * u.TeV]
)
analysis.datasets[1].models[0].spectral_model.plot_error(
    sed_type="e2dnde", energy_bounds=[70 * u.MeV, 10 * u.TeV]
)

plt.grid(which="both")
plt.legend()